In [2]:
import os
import h5py
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
data_dir='Outputs/'

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
   sys.path.append(module_path)


# make sure we're dealing with the most recent version of any code we're using
%load_ext autoreload
%autoreload 2

In [6]:
ATL03_file=glob('/home/jovyan/crackup/Outputs_GeorgeVI/*ATL03*.h5')
ATL06_file=glob('/home/jovyan/crackup/Outputs_GeorgeVI/*ATL06*.h5')
%matplotlib widget
beamNum = [1,3,5]

f = h5py.File(ATL03_file[0], 'r')  # keep it open
beam = [k for k in f.keys() if k.startswith('gt')]

f6 = h5py.File(ATL06_file[0], 'r')
beam6 = [k for k in f6.keys() if k.startswith('gt')]

lookfor = ['delta_time','h_li','h_li_sigma','latitude','longitude','segment_id','sigma_geo_h']
fig, ax = plt.subplots(nrows=3, ncols=1, sharex=True,figsize=(10,8))
data = {}
data6 = {}

for ii,p in enumerate(beamNum):

# Extract data from ATL03
   data[beam[p]] = {}
   data[beam[p]]['heights'] = {}

   for key,val in f[beam[p]]['heights'].items():
       data[beam[p]]['heights'][key] = val[:]

# Assign flag based on confidence level
   #-- 0=Land; 1=Ocean; 2=SeaIce; 3=LandIce; 4=InlandWater
   conf = data[beam[p]]['heights']['signal_conf_ph'][:,3] #choose column three for LandIce
    #-- background and buffer photons
   bg = np.nonzero((conf == 0) | (conf == 1))
   lc = np.nonzero(conf == 2)
   mc = np.nonzero(conf == 3)
   hc = np.nonzero(conf == 4)

# Extract data from ATL06
   data6[beam6[p]] = {}
   data6[beam6[p]]['land_ice_segments'] = {}

#    print('what',f6[beam6[p]]['land_ice_segments'].keys())
   for key,val in f6[beam6[p]]['land_ice_segments'].items():
       if key in lookfor:
           data6[beam6[p]]['land_ice_segments'][key] = val[:]

#  Shorten variable names
   D3x = data[beam[p]]['heights']['lat_ph']
   D3y = data[beam[p]]['heights']['h_ph']
   D6x = data6[beam[p]]['land_ice_segments']['latitude']
   D6y = data6[beam[p]]['land_ice_segments']['h_li']

#     %matplotlib widget
#    ax[ii].plot(data[beam[p]]['heights']['delta_time'][conf>=2],data[beam[p]]['heights']['h_ph'][conf>=2],'k.',markersize=0.1)
#    ax[ii].plot(D3x[bg],D3y[bg],marker='.',lw=0,markersize=0.1,color='gray',label='Background')
   ax[ii].plot(D3x[lc],D3y[lc],marker='.',lw=0,markersize=0.1,color='darkorange',label='ALT03 - Low Confidence')
   ax[ii].plot(D3x[mc],D3y[mc],marker='.',lw=0,markersize=0.2,color='mediumseagreen',label='ALT03 - Medium Confidence')
   ax[ii].plot(D3x[hc],D3y[hc],marker='.',lw=0,markersize=0.3,color='darkorchid',label='ATL03 - High Confidence')
   ax[ii].plot(D6x,D6y,'r:',lw=1.5,label='ATL06 Surface')


#  labels & limits
   ax[ii].set_title('Beam ' + str(ii+1))
#    ax[ii].set_ylim(5,30)
   ax[ii].set_xlim(-71.525,-71.36)
   ax[ii].set_ylabel('Elevation (m)')
    
ax[ii].set_xlabel('Latitude (dec degrees)')
lgd = ax[0].legend(loc=0,frameon=False)
lgd.get_frame().set_alpha(1.0)
for line in lgd.get_lines():
    line.set_linewidth(6)
plt.show()
f.close()
f6.close()


FigureCanvasNbAgg()